In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import joblib
import os
import glob
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RandomizedSearchCV

## SVM（フィルタ後の先頭30フレームの平均）

In [2]:
# 処理するフォルダのパス
folder_path = './output_moving_filtered_data'

# フォルダ内のすべてのCSVファイルを取得
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
labels = [0,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,1,1,1,1,1,0,1,0,0,0]
i = 0
i = 0
# 各ファイルに対して処理を適用
result_list = []  # 平均値を保存するリスト

for file_name in csv_files:
    file_path = os.path.join(folder_path, file_name)
    
    # データの読み込み
    df = pd.read_csv(file_path, low_memory=False)
    
    # `question1`から`question10`のみをフィルタリング
    filtered_df = df[df["label"].str.match(r'^question(10|[1-9])$', na=False)]
    
    # Confidenceが0.90以上の行を抽出
    filtered_df = filtered_df[filtered_df["Confidence"] >= 0.90]
    
    # `Confidence`列と`AU`で始まる列を抽出
    columns_to_extract = ["label", "Confidence"] + [col for col in filtered_df.columns if col.startswith("AU")]
    filtered_df = filtered_df[columns_to_extract]
    
    # 順序を保つためのカスタム順序のカテゴリを定義
    filtered_df["label"] = pd.Categorical(
        filtered_df["label"],
        categories=[f"question{i}" for i in range(1, 11)],
        ordered=True
    )
    
    # ラベルごとに最初の30フレームを取得
    first_30_frames_per_label = (
        filtered_df.groupby("label")
        .apply(lambda group: group.head(30))
        .reset_index(drop=True)  # グループ化インデックスをリセット
    )
    
    # 平均を計算
    mean_values_per_label = first_30_frames_per_label.groupby("label").mean(numeric_only=True)
    
    # NaNを含む行を削除
    mean_values_per_label = mean_values_per_label.dropna(axis=0)
    
    # Yを追加
    mean_values_per_label["Y"] = labels[i]
    i = i + 1
    
    # 結果をリストに保存
    result_list.append((file_name, mean_values_per_label))

# 各ファイルの結果を出力
for file_name, mean_values in result_list:
    print(f"Results for {file_name}:")
    print(mean_values)
    print()

Results for filtered_ID30_smoothed.csv:
            Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                      
question1     0.995516  0.586705  0.278403  0.464329  0.311001  0.185493   
question2     0.995715  0.642548  0.393834  0.514342  0.303340  0.173015   
question3     0.996076  0.653089  0.351344  0.467197  0.328368  0.162328   
question4     0.996124  0.611314  0.306931  0.452627  0.335895  0.182368   
question5     0.995153  0.617566  0.277745  0.575782  0.326813  0.310574   
question6     0.994046  0.619970  0.368858  0.507259  0.443115  0.232560   
question7     0.994499  0.640611  0.342359  0.465248  0.412585  0.208544   
question8     0.996331  0.572323  0.331490  0.458027  0.314261  0.291824   
question9     0.997603  0.716356  0.389402  0.485613  0.367574  0.223056   
question10    0.996228  0.739683  0.410449  0.465391  0.374358  0.198075   

                AU09      AU10      AU12      A

            Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                      
question1     0.999797  0.663449  0.300578  0.175094  0.397135  0.087894   
question2     0.998309  0.613413  0.488225  0.182503  0.466726  0.186421   
question3     0.995119  0.706113  0.342605  0.132918  0.496969  0.075051   
question4     0.993577  0.620753  0.522889  0.112564  0.599509  0.071535   
question5     0.996099  0.591205  0.511268  0.158214  0.631687  0.076867   
question6     0.998682  0.606885  0.503284  0.106865  0.606659  0.074689   
question7     0.999010  0.575101  0.364279  0.241101  0.522647  0.054403   
question8     0.996254  0.637121  0.357319  0.098621  0.524664  0.075818   
question10    0.998439  0.602762  0.296254  0.144828  0.510599  0.054888   

                AU09      AU10      AU12      AU14      AU15      AU17  \
label                                                                    
question1   0.1

            Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                      
question1     0.995326  0.540144  0.487645  0.362183  0.370982  0.223776   
question2     0.997699  0.610590  0.659309  0.298785  0.416073  0.178938   
question3     0.997732  0.656399  0.697681  0.358727  0.403803  0.167546   
question5     0.999241  0.619400  0.180826  0.684102  0.366452  0.158987   
question6     0.998153  0.624267  0.401849  0.424342  0.390251  0.163629   
question7     0.999197  0.574999  0.244682  0.490499  0.368001  0.155968   
question8     0.997764  0.584155  0.395723  0.448220  0.398376  0.180634   
question9     0.998324  0.581483  0.426760  0.387990  0.381731  0.156377   
question10    0.998615  0.616858  0.713556  0.283535  0.386161  0.209556   

                AU09      AU10      AU12      AU14      AU15      AU17  \
label                                                                    
question1   0.3

In [3]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

# 削除する列を指定
columns_to_drop = ['Confidence','label', 'AU07', 'AU11', 'AU20']

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # SVMモデルの初期化
    model = SVC(kernel='rbf', probability=True)# 'rbf'カーネルを使用（非線形データの場合）
    model.fit(X_train_combined, y_train_combined)
    
    # 予測
    y_pred = model.predict(X_test)
    y_pred_prob = model.predict_proba(X_test)[:, 1]  # クラス1の確率
    
    # 精度計算
    accuracy = accuracy_score(y_test, y_pred) * 100  # 正答率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")  # 小数点以下2桁で%表示
    print(f"正解ラベル: {y_test.tolist()}")  # 正解ラベルをリストとして表示
    print(f"予測ラベル: {y_pred.tolist()}")  # 予測ラベルをリストとして表示
    print(f"クラス1の確率: {y_pred_prob}")  # 各テストデータ点のクラス1の確率

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")


===== テストデータ: ファイル 1/31 =====
ファイル名: filtered_ID30_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.16084324 0.28977622 0.1783056  0.0993342  0.31020042 0.40426938
 0.25933622 0.25276015 0.37098533 0.39965474]

===== テストデータ: ファイル 2/31 =====
ファイル名: filtered_ID31_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.03724464 0.01923186 0.02412064 0.03612739 0.02233272 0.03271893
 0.02192182 0.05530343 0.01640448 0.01315717]

===== テストデータ: ファイル 3/31 =====
ファイル名: filtered_ID32_smoothed.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.01676196 0.02228965 0.03830878 0.04510082 0.06776473 0.01846246
 0.03918867 0.07946303 0.00383252]

===== テストデータ: ファイル 4/31 =====
ファイル名: filtered_ID33_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.07491655 0.10150442 0.148

In [3]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report

# 削除する列を指定
columns_to_drop = ['Confidence', 'label', 'AU07', 'AU11', 'AU20']

# グリッドサーチ用のパラメータ設定
param_grid = {
    'C': [0.1, 1, 10, 100],         # 正則化パラメータ
    'gamma': [1, 0.1, 0.01, 0.001], # RBFカーネルのgamma
    'kernel': ['rbf']               # カーネルはRBF
}

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # SVMモデルのグリッドサーチ
    model = SVC(probability=True)
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=3, n_jobs=-1)
    grid_search.fit(X_train_combined, y_train_combined)
    
    # 最適なモデルを取得
    best_model = grid_search.best_estimator_
    print(f"最適パラメータ: {grid_search.best_params_}")
    
    # テストデータで予測
    y_pred = best_model.predict(X_test)
    y_pred_prob = best_model.predict_proba(X_test)[:, 1]  # クラス1の確率
    
    # 精度計算
    accuracy = accuracy_score(y_test, y_pred) * 100  # 正答率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")
    print(f"正解ラベル: {y_test.tolist()}")
    print(f"予測ラベル: {y_pred.tolist()}")
    print(f"クラス1の確率: {y_pred_prob}")
    
# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")



===== テストデータ: ファイル 1/31 =====
最適パラメータ: {'C': 10, 'gamma': 0.1, 'kernel': 'rbf'}
ファイル名: filtered_ID30_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.1932921  0.30764133 0.21802236 0.1495235  0.25350018 0.30675722
 0.27697521 0.22153516 0.33098947 0.37168603]

===== テストデータ: ファイル 2/31 =====
最適パラメータ: {'C': 100, 'gamma': 0.1, 'kernel': 'rbf'}
ファイル名: filtered_ID31_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.04902303 0.02956756 0.00778697 0.06188145 0.02295439 0.07008094
 0.02190989 0.12964635 0.02959071 0.01853614]

===== テストデータ: ファイル 3/31 =====
最適パラメータ: {'C': 100, 'gamma': 0.01, 'kernel': 'rbf'}
ファイル名: filtered_ID32_smoothed.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.08998875 0.06424748 0.0717503  0.07267266 0.16981133 0.07433743
 0.07470185 0.11594751 0.03008308]

===== テストデータ: ファイル 4/31 =====
最適パラメータ

最適パラメータ: {'C': 100, 'gamma': 0.01, 'kernel': 'rbf'}
ファイル名: filtered_ID69_smoothed.csv
予測率: 90.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.69237399 0.3930582  0.33414789 0.17061345 0.0600116  0.37312882
 0.08999601 0.5349132  0.17301248 0.24353678]

===== テストデータ: ファイル 28/31 =====
最適パラメータ: {'C': 100, 'gamma': 0.01, 'kernel': 'rbf'}
ファイル名: filtered_ID70_smoothed.csv
予測率: 44.44%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 1, 0, 0, 1, 0, 1, 1]
クラス1の確率: [0.15645707 0.4192395  0.59662217 0.27803862 0.1524507  0.64465796
 0.5217432  0.79234081 0.73158459]

===== テストデータ: ファイル 29/31 =====
最適パラメータ: {'C': 100, 'gamma': 0.1, 'kernel': 'rbf'}
ファイル名: filtered_ID71_smoothed.csv
予測率: 80.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [1, 0, 0, 1, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.70832659 0.2089796  0.31220816 0.67366969 0.39890854 0.29578856
 0.42680134 0.33609194 0.38734101 0.20551605]

===== テストデータ: ファイル 30/31 =====
最適パラメータ: {'C': 100, 'gamma': 0.01,

## SVM（フィルタ後の先頭10フレームの平均）

In [4]:
# 処理するフォルダのパス
folder_path = './output_moving_filtered_data'

# フォルダ内のすべてのCSVファイルを取得
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
labels = [0,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,1,1,1,1,1,0,1,0,0,0]
i = 0
i = 0
# 各ファイルに対して処理を適用
result_list = []  # 平均値を保存するリスト

for file_name in csv_files:
    file_path = os.path.join(folder_path, file_name)
    
    # データの読み込み
    df = pd.read_csv(file_path, low_memory=False)
    
    # `question1`から`question10`のみをフィルタリング
    filtered_df = df[df["label"].str.match(r'^question(10|[1-9])$', na=False)]
    
    # Confidenceが0.90以上の行を抽出
    filtered_df = filtered_df[filtered_df["Confidence"] >= 0.90]
    
    # `Confidence`列と`AU`で始まる列を抽出
    columns_to_extract = ["label", "Confidence"] + [col for col in filtered_df.columns if col.startswith("AU")]
    filtered_df = filtered_df[columns_to_extract]
    
    # 順序を保つためのカスタム順序のカテゴリを定義
    filtered_df["label"] = pd.Categorical(
        filtered_df["label"],
        categories=[f"question{i}" for i in range(1, 11)],
        ordered=True
    )
    
    # ラベルごとに最初の10フレームを取得
    first_10_frames_per_label = (
        filtered_df.groupby("label")
        .apply(lambda group: group.head(10))
        .reset_index(drop=True)  # グループ化インデックスをリセット
    )
    
    # 平均を計算
    mean_values_per_label = first_10_frames_per_label.groupby("label").mean(numeric_only=True)
    
    # NaNを含む行を削除
    mean_values_per_label = mean_values_per_label.dropna(axis=0)
    
    # Yを追加
    mean_values_per_label["Y"] = labels[i]
    i = i + 1
    
    # 結果をリストに保存
    result_list.append((file_name, mean_values_per_label))

# 各ファイルの結果を出力
for file_name, mean_values in result_list:
    print(f"Results for {file_name}:")
    print(mean_values)
    print()

Results for filtered_ID30_smoothed.csv:
            Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                      
question1     0.994253  0.563476  0.291621  0.407823  0.340137  0.208585   
question2     0.995213  0.679677  0.489989  0.533290  0.270572  0.194329   
question3     0.996800  0.655168  0.335807  0.474354  0.322668  0.164400   
question4     0.996812  0.600369  0.304642  0.479000  0.312973  0.185587   
question5     0.995148  0.597816  0.281193  0.577127  0.334294  0.321020   
question6     0.995141  0.618017  0.380514  0.512854  0.444987  0.243883   
question7     0.994641  0.674597  0.359317  0.506275  0.382306  0.228034   
question8     0.997165  0.563294  0.317174  0.466524  0.315580  0.271764   
question9     0.998061  0.696176  0.394051  0.474845  0.374721  0.229086   
question10    0.996726  0.721274  0.421932  0.451504  0.436006  0.192772   

                AU09      AU10      AU12      A

            Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                      
question2     0.998475  0.774619  0.703730  0.128377  0.404937  0.103893   
question3     0.998389  0.802554  0.656277  0.318188  0.314823  0.110575   
question4     0.999495  0.786576  0.570987  0.362731  0.356207  0.106907   
question5     0.999506  0.775146  0.633499  0.377597  0.333924  0.116994   
question6     0.999330  0.801850  0.647985  0.363255  0.374534  0.109995   
question7     0.998592  0.806244  0.731101  0.375919  0.327706  0.145853   
question8     0.999322  0.794013  0.688489  0.241081  0.436412  0.109898   
question9     0.998824  0.627024  0.422349  0.182895  0.257014  0.132412   
question10    0.999369  0.666210  0.476638  0.231186  0.272389  0.112288   

                AU09      AU10      AU12      AU14      AU15      AU17  \
label                                                                    
question2   0.1

            Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                      
question1     0.986897  0.515889  0.320827  0.498142  0.402086  0.131150   
question2     0.998291  0.762448  0.510442  0.323292  0.348063  0.144675   
question3     0.998069  0.581119  0.348987  0.421618  0.327304  0.172630   
question4     0.998514  0.697572  0.427292  0.338921  0.346921  0.219894   
question5     0.993900  0.651011  0.379904  0.265072  0.366228  0.210694   
question6     0.997559  0.734107  0.459258  0.376157  0.303736  0.411712   
question7     0.996826  0.697459  0.483100  0.245377  0.373801  0.161121   
question8     0.995752  0.625220  0.373010  0.409359  0.333614  0.331951   
question9     0.997245  0.699948  0.410207  0.485947  0.289984  0.451468   
question10    0.998655  0.630636  0.420462  0.421223  0.332868  0.272557   

                AU09      AU10      AU12      AU14      AU15      AU17  \
label        

In [5]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

# 削除する列を指定
columns_to_drop = ['Confidence','label', 'AU07', 'AU11', 'AU20']

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # SVMモデルの初期化
    model = SVC(kernel='rbf', probability=True)# 'rbf'カーネルを使用（非線形データの場合）
    model.fit(X_train_combined, y_train_combined)
    
    # 予測
    y_pred = model.predict(X_test)
    y_pred_prob = model.predict_proba(X_test)[:, 1]  # クラス1の確率
    
    # 精度計算
    accuracy = accuracy_score(y_test, y_pred) * 100  # 正答率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")  # 小数点以下2桁で%表示
    print(f"正解ラベル: {y_test.tolist()}")  # 正解ラベルをリストとして表示
    print(f"予測ラベル: {y_pred.tolist()}")  # 予測ラベルをリストとして表示
    print(f"クラス1の確率: {y_pred_prob}")  # 各テストデータ点のクラス1の確率

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")


===== テストデータ: ファイル 1/31 =====
ファイル名: filtered_ID30_smoothed.csv
予測率: 80.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 1, 1]
クラス1の確率: [0.33684023 0.49423932 0.21514841 0.12485073 0.3705252  0.49425856
 0.33602198 0.26178662 0.57624034 0.64703429]

===== テストデータ: ファイル 2/31 =====
ファイル名: filtered_ID31_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.14475389 0.06827556 0.08730579 0.10643815 0.05820189 0.06313167
 0.07551409 0.18861542 0.07824743 0.0309359 ]

===== テストデータ: ファイル 3/31 =====
ファイル名: filtered_ID32_smoothed.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.04472913 0.04121689 0.03975803 0.05461161 0.0539971  0.02834164
 0.10012492 0.09058596 0.00926367]

===== テストデータ: ファイル 4/31 =====
ファイル名: filtered_ID33_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.21987926 0.14097344 0.1647

In [5]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report

# 削除する列を指定
columns_to_drop = ['Confidence', 'label', 'AU07', 'AU11', 'AU20']

# グリッドサーチ用のパラメータ設定
param_grid = {
    'C': [0.1, 1, 10, 100],         # 正則化パラメータ
    'gamma': [1, 0.1, 0.01, 0.001], # RBFカーネルのgamma
    'kernel': ['rbf']               # カーネルはRBF
}

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # SVMモデルのグリッドサーチ
    model = SVC(probability=True)
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=3, n_jobs=-1)
    grid_search.fit(X_train_combined, y_train_combined)
    
    # 最適なモデルを取得
    best_model = grid_search.best_estimator_
    print(f"最適パラメータ: {grid_search.best_params_}")
    
    # テストデータで予測
    y_pred = best_model.predict(X_test)
    y_pred_prob = best_model.predict_proba(X_test)[:, 1]  # クラス1の確率
    
    # 精度計算
    accuracy = accuracy_score(y_test, y_pred) * 100  # 正答率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")
    print(f"正解ラベル: {y_test.tolist()}")
    print(f"予測ラベル: {y_pred.tolist()}")
    print(f"クラス1の確率: {y_pred_prob}")
    
# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")



===== テストデータ: ファイル 1/31 =====
最適パラメータ: {'C': 100, 'gamma': 0.1, 'kernel': 'rbf'}
ファイル名: filtered_ID30_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.24100363 0.36790229 0.16968523 0.08849649 0.26124425 0.29214743
 0.29941369 0.19344116 0.39488217 0.45633561]

===== テストデータ: ファイル 2/31 =====
最適パラメータ: {'C': 100, 'gamma': 0.01, 'kernel': 'rbf'}
ファイル名: filtered_ID31_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.15817007 0.07944082 0.02931928 0.18431586 0.12955465 0.12271337
 0.09549481 0.16984284 0.13216153 0.08042621]

===== テストデータ: ファイル 3/31 =====
最適パラメータ: {'C': 100, 'gamma': 0.01, 'kernel': 'rbf'}
ファイル名: filtered_ID32_smoothed.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.12971915 0.07092511 0.07352959 0.0783361  0.11753119 0.07539093
 0.12031273 0.09041145 0.03991694]

===== テストデータ: ファイル 4/31 =====
最適パラメ

最適パラメータ: {'C': 100, 'gamma': 0.01, 'kernel': 'rbf'}
ファイル名: filtered_ID69_smoothed.csv
予測率: 90.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.55636862 0.39172087 0.43775572 0.31926482 0.15725678 0.38759617
 0.11458404 0.40129311 0.23111576 0.40587162]

===== テストデータ: ファイル 28/31 =====
最適パラメータ: {'C': 10, 'gamma': 0.1, 'kernel': 'rbf'}
ファイル名: filtered_ID70_smoothed.csv
予測率: 33.33%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 1, 0, 1, 1]
クラス1の確率: [0.29455055 0.48649185 0.49373454 0.17046808 0.13536304 0.68761973
 0.55408682 0.73566857 0.70205031]

===== テストデータ: ファイル 29/31 =====
最適パラメータ: {'C': 100, 'gamma': 0.01, 'kernel': 'rbf'}
ファイル名: filtered_ID71_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.42444354 0.26531769 0.22018873 0.34811608 0.26616415 0.15392814
 0.28870883 0.27858427 0.2140174  0.15290994]

===== テストデータ: ファイル 30/31 =====
最適パラメータ: {'C': 100, 'gamma': 0.01,

## SVM（フィルタ後の先頭30フレームの標準偏差）

In [6]:
# 処理するフォルダのパス
folder_path = './output_moving_filtered_data'

# フォルダ内のすべてのCSVファイルを取得
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
labels = [0,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,1,1,1,1,1,0,1,0,0,0]
i = 0
i = 0
# 各ファイルに対して処理を適用
result_list = []  # 平均値を保存するリスト

for file_name in csv_files:
    file_path = os.path.join(folder_path, file_name)
    
    # データの読み込み
    df = pd.read_csv(file_path, low_memory=False)
    
    # `question1`から`question10`のみをフィルタリング
    filtered_df = df[df["label"].str.match(r'^question(10|[1-9])$', na=False)]
    
    # Confidenceが0.90以上の行を抽出
    filtered_df = filtered_df[filtered_df["Confidence"] >= 0.90]
    
    # `Confidence`列と`AU`で始まる列を抽出
    columns_to_extract = ["label", "Confidence"] + [col for col in filtered_df.columns if col.startswith("AU")]
    filtered_df = filtered_df[columns_to_extract]
    
    # 順序を保つためのカスタム順序のカテゴリを定義
    filtered_df["label"] = pd.Categorical(
        filtered_df["label"],
        categories=[f"question{i}" for i in range(1, 11)],
        ordered=True
    )
    
    # ラベルごとに最初の30フレームを取得
    first_30_frames_per_label = (
        filtered_df.groupby("label")
        .apply(lambda group: group.head(30))
        .reset_index(drop=True)  # グループ化インデックスをリセット
    )
    
    # 標準偏差を計算
    std_values_per_label = first_30_frames_per_label.groupby("label").std(numeric_only=True)

    # NaNを含む行を削除
    std_values_per_label = std_values_per_label.dropna(axis=0)
    
    # Yを追加
    std_values_per_label["Y"] = labels[i]
    i = i + 1
    
    # 結果をリストに保存
    result_list.append((file_name, std_values_per_label))

# 各ファイルの結果を出力
for file_name, mean_values in result_list:
    print(f"Results for {file_name}:")
    print(mean_values)
    print()

Results for filtered_ID30_smoothed.csv:
            Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                      
question1     0.001466  0.025033  0.023155  0.053183  0.023770  0.020858   
question2     0.001270  0.041890  0.075525  0.024075  0.028638  0.017322   
question3     0.001708  0.015978  0.029135  0.021304  0.016300  0.013930   
question4     0.001270  0.022869  0.011865  0.032367  0.027939  0.009538   
question5     0.001197  0.018880  0.036619  0.035231  0.017124  0.031189   
question6     0.001881  0.023875  0.014404  0.023272  0.024397  0.018919   
question7     0.002001  0.031089  0.017664  0.052384  0.042433  0.015764   
question8     0.001401  0.012390  0.038962  0.031037  0.008042  0.021269   
question9     0.001172  0.031922  0.028190  0.022848  0.031537  0.010618   
question10    0.001385  0.021430  0.019002  0.038693  0.048220  0.007847   

                AU09      AU10      AU12      A

            Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                      
question1     0.000056  0.009618  0.028260  0.012581  0.016106  0.020242   
question2     0.000449  0.034466  0.024327  0.052048  0.030721  0.053032   
question3     0.001533  0.014374  0.033090  0.015698  0.035024  0.005674   
question4     0.001772  0.029188  0.038147  0.009578  0.020594  0.007930   
question5     0.001203  0.031426  0.017179  0.012046  0.009744  0.004272   
question6     0.000350  0.035870  0.038249  0.009545  0.007986  0.003866   
question7     0.000269  0.017203  0.022636  0.056831  0.057583  0.003498   
question8     0.001678  0.019623  0.026455  0.021017  0.026839  0.002686   
question10    0.000483  0.034667  0.047369  0.007989  0.068297  0.004456   

                AU09      AU10      AU12      AU14      AU15      AU17  \
label                                                                    
question1   0.0

            Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                      
question1     0.000620  0.033630  0.029076  0.042905  0.023929  0.033291   
question2     0.003943  0.050951  0.050071  0.082208  0.020488  0.039662   
question3     0.009322  0.052634  0.039735  0.060323  0.020753  0.037847   
question4     0.000438  0.076899  0.072969  0.099328  0.036146  0.005414   
question5     0.000477  0.021521  0.041965  0.055464  0.035054  0.018451   
question6     0.000657  0.037212  0.051184  0.069576  0.019174  0.022720   
question7     0.000633  0.044111  0.036444  0.054153  0.031572  0.023376   
question8     0.000523  0.060790  0.065995  0.091470  0.034498  0.010904   
question9     0.000450  0.059023  0.083248  0.076410  0.018851  0.015996   
question10    0.000690  0.078620  0.098683  0.154086  0.027941  0.003882   

                AU09      AU10      AU12      AU14      AU15      AU17  \
label        

In [7]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

# 削除する列を指定
columns_to_drop = ['Confidence','label', 'AU07', 'AU11', 'AU20']

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # SVMモデルの初期化
    model = SVC(kernel='rbf', probability=True)# 'rbf'カーネルを使用（非線形データの場合）
    model.fit(X_train_combined, y_train_combined)
    
    # 予測
    y_pred = model.predict(X_test)
    y_pred_prob = model.predict_proba(X_test)[:, 1]  # クラス1の確率
    
    # 精度計算
    accuracy = accuracy_score(y_test, y_pred) * 100  # 正答率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")  # 小数点以下2桁で%表示
    print(f"正解ラベル: {y_test.tolist()}")  # 正解ラベルをリストとして表示
    print(f"予測ラベル: {y_pred.tolist()}")  # 予測ラベルをリストとして表示
    print(f"クラス1の確率: {y_pred_prob}")  # 各テストデータ点のクラス1の確率

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")


===== テストデータ: ファイル 1/31 =====
ファイル名: filtered_ID30_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.40159792 0.28119345 0.41230813 0.30492094 0.23030948 0.24207024
 0.30934844 0.29214421 0.33190699 0.38785553]

===== テストデータ: ファイル 2/31 =====
ファイル名: filtered_ID31_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.28193197 0.26665577 0.42988719 0.36993489 0.32361774 0.42411549
 0.32982141 0.44216859 0.3840928  0.44266837]

===== テストデータ: ファイル 3/31 =====
ファイル名: filtered_ID32_smoothed.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.34481142 0.2160354  0.20240728 0.26846965 0.35081476 0.24651832
 0.22137729 0.24300365 0.24322935]

===== テストデータ: ファイル 4/31 =====
ファイル名: filtered_ID33_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.22364131 0.30500326 0.303

In [7]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report

# 削除する列を指定
columns_to_drop = ['Confidence', 'label', 'AU07', 'AU11', 'AU20']

# グリッドサーチ用のパラメータ設定
param_grid = {
    'C': [0.1, 1, 10, 100],         # 正則化パラメータ
    'gamma': [1, 0.1, 0.01, 0.001], # RBFカーネルのgamma
    'kernel': ['rbf']               # カーネルはRBF
}

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # SVMモデルのグリッドサーチ
    model = SVC(probability=True)
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=3, n_jobs=-1)
    grid_search.fit(X_train_combined, y_train_combined)
    
    # 最適なモデルを取得
    best_model = grid_search.best_estimator_
    print(f"最適パラメータ: {grid_search.best_params_}")
    
    # テストデータで予測
    y_pred = best_model.predict(X_test)
    y_pred_prob = best_model.predict_proba(X_test)[:, 1]  # クラス1の確率
    
    # 精度計算
    accuracy = accuracy_score(y_test, y_pred) * 100  # 正答率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")
    print(f"正解ラベル: {y_test.tolist()}")
    print(f"予測ラベル: {y_pred.tolist()}")
    print(f"クラス1の確率: {y_pred_prob}")
    
# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")



===== テストデータ: ファイル 1/31 =====
最適パラメータ: {'C': 0.1, 'gamma': 1, 'kernel': 'rbf'}
ファイル名: filtered_ID30_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.4491861  0.32886728 0.41065678 0.36518542 0.30104558 0.32952047
 0.38597676 0.38331876 0.3401766  0.3752289 ]

===== テストデータ: ファイル 2/31 =====
最適パラメータ: {'C': 10, 'gamma': 1, 'kernel': 'rbf'}
ファイル名: filtered_ID31_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.36824603 0.36340557 0.36417146 0.33087745 0.30282404 0.44777851
 0.30238092 0.41869142 0.37702053 0.45882743]

===== テストデータ: ファイル 3/31 =====
最適パラメータ: {'C': 0.1, 'gamma': 1, 'kernel': 'rbf'}
ファイル名: filtered_ID32_smoothed.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.42101474 0.31248799 0.25934056 0.31771767 0.33471907 0.2711966
 0.26977881 0.30941855 0.2930034 ]

===== テストデータ: ファイル 4/31 =====
最適パラメータ: {'C': 

最適パラメータ: {'C': 0.1, 'gamma': 1, 'kernel': 'rbf'}
ファイル名: filtered_ID69_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.37957764 0.27028987 0.36992379 0.3177719  0.36668569 0.33376635
 0.27346677 0.34276174 0.33754413 0.30312711]

===== テストデータ: ファイル 28/31 =====
最適パラメータ: {'C': 0.1, 'gamma': 1, 'kernel': 'rbf'}
ファイル名: filtered_ID70_smoothed.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.4857657  0.39160702 0.35157216 0.43694239 0.34378349 0.33799653
 0.54712619 0.43229151 0.50564085]

===== テストデータ: ファイル 29/31 =====
最適パラメータ: {'C': 0.1, 'gamma': 1, 'kernel': 'rbf'}
ファイル名: filtered_ID71_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.44081382 0.2964053  0.34250492 0.2412296  0.47506285 0.29039655
 0.29207314 0.29326584 0.44318888 0.27936582]

===== テストデータ: ファイル 30/31 =====
最適パラメータ: {'C': 100, 'gamma': 0.1, 'kernel

## SVM（フィルタ後の先頭10フレームの標準偏差）

In [2]:
# 処理するフォルダのパス
folder_path = './output_moving_filtered_data'

# フォルダ内のすべてのCSVファイルを取得
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
labels = [0,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,1,1,1,1,1,0,1,0,0,0]
i = 0
i = 0
# 各ファイルに対して処理を適用
result_list = []  # 平均値を保存するリスト

for file_name in csv_files:
    file_path = os.path.join(folder_path, file_name)
    
    # データの読み込み
    df = pd.read_csv(file_path, low_memory=False)
    
    # `question1`から`question10`のみをフィルタリング
    filtered_df = df[df["label"].str.match(r'^question(10|[1-9])$', na=False)]
    
    # Confidenceが0.90以上の行を抽出
    filtered_df = filtered_df[filtered_df["Confidence"] >= 0.90]
    
    # `Confidence`列と`AU`で始まる列を抽出
    columns_to_extract = ["label", "Confidence"] + [col for col in filtered_df.columns if col.startswith("AU")]
    filtered_df = filtered_df[columns_to_extract]
    
    # 順序を保つためのカスタム順序のカテゴリを定義
    filtered_df["label"] = pd.Categorical(
        filtered_df["label"],
        categories=[f"question{i}" for i in range(1, 11)],
        ordered=True
    )
    
    # ラベルごとに最初の30フレームを取得
    first_30_frames_per_label = (
        filtered_df.groupby("label")
        .apply(lambda group: group.head(10))
        .reset_index(drop=True)  # グループ化インデックスをリセット
    )
    
    # 標準偏差を計算
    std_values_per_label = first_30_frames_per_label.groupby("label").std(numeric_only=True)

    # NaNを含む行を削除
    std_values_per_label = std_values_per_label.dropna(axis=0)
    
    # Yを追加
    std_values_per_label["Y"] = labels[i]
    i = i + 1
    
    # 結果をリストに保存
    result_list.append((file_name, std_values_per_label))

# 各ファイルの結果を出力
for file_name, mean_values in result_list:
    print(f"Results for {file_name}:")
    print(mean_values)
    print()

Results for filtered_ID30_smoothed.csv:
            Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                      
question1     0.001121  0.012138  0.009317  0.023847  0.005584  0.012798   
question2     0.001382  0.023488  0.036884  0.028307  0.013806  0.007775   
question3     0.000877  0.009082  0.017248  0.013338  0.016935  0.007002   
question4     0.000845  0.016595  0.016401  0.013096  0.020036  0.009922   
question5     0.001609  0.005863  0.030660  0.044703  0.004662  0.023892   
question6     0.001885  0.002587  0.010399  0.033491  0.015164  0.015248   
question7     0.001011  0.010774  0.015984  0.033539  0.029998  0.006724   
question8     0.000721  0.007721  0.018937  0.046402  0.007213  0.004194   
question9     0.000666  0.015087  0.020192  0.007193  0.036285  0.006396   
question10    0.001114  0.006376  0.011720  0.019317  0.017930  0.006203   

                AU09      AU10      AU12      A

            Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                      
question2     0.000319  0.006680  0.017305  0.010684  0.013701  0.011816   
question3     0.000238  0.050899  0.028056  0.022083  0.002575  0.009814   
question4     0.000066  0.016010  0.016583  0.018330  0.012876  0.024784   
question6     0.000068  0.010926  0.007183  0.022033  0.006648  0.015874   
question7     0.000215  0.015538  0.013323  0.019948  0.003824  0.015066   
question8     0.000096  0.020129  0.037334  0.045764  0.007780  0.016931   
question9     0.000131  0.017342  0.018210  0.006391  0.003689  0.008941   
question10    0.001251  0.006701  0.009923  0.032444  0.027921  0.005662   

                AU09      AU10      AU12      AU14      AU15      AU17  \
label                                                                    
question2   0.022289  0.020787  0.021665  0.021254  0.031606  0.023298   
question3   0.008

            Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                      
question1     0.000493  0.029068  0.050211  0.053161  0.007932  0.026421   
question2     0.000015  0.004123  0.023947  0.020348  0.025905  0.009228   
question3     0.000030  0.004521  0.006781  0.016023  0.014177  0.004135   
question4     0.000017  0.005964  0.019684  0.033870  0.054067  0.008635   
question5     0.000014  0.005978  0.025597  0.010190  0.002851  0.017309   
question6     0.000025  0.018477  0.046551  0.019743  0.003252  0.004906   
question7     0.000008  0.004504  0.017954  0.034458  0.011781  0.010656   
question8     0.000150  0.007867  0.021005  0.023411  0.004379  0.006319   
question9     0.000032  0.006783  0.021022  0.038785  0.011777  0.014282   
question10    0.000098  0.015437  0.010926  0.018615  0.016620  0.020073   

                AU09      AU10      AU12      AU14      AU15      AU17  \
label        

            Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                      
question1     0.000794  0.034469  0.016120  0.040530  0.015332  0.012351   
question2     0.002203  0.023590  0.030605  0.026931  0.012040  0.005364   
question3     0.002005  0.020868  0.026518  0.031446  0.018515  0.007496   
question4     0.002759  0.015361  0.015747  0.014157  0.008742  0.009794   
question5     0.003626  0.015611  0.035572  0.031287  0.012545  0.013234   
question6     0.002346  0.010346  0.061792  0.036605  0.016582  0.007528   
question7     0.003021  0.020120  0.023463  0.030837  0.017783  0.008542   
question8     0.001761  0.028255  0.051494  0.025792  0.022584  0.010919   
question9     0.001957  0.015133  0.022828  0.022708  0.016317  0.003571   
question10    0.002301  0.045815  0.074245  0.051050  0.012810  0.010260   

                AU09      AU10      AU12      AU14      AU15      AU17  \
label        

            Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                      
question1     0.002264  0.014793  0.019376  0.011130  0.006167  0.009583   
question2     0.006205  0.018662  0.014029  0.046820  0.019926  0.005449   
question3     0.003995  0.051291  0.074559  0.069948  0.008452  0.014615   
question4     0.005631  0.103757  0.101007  0.060363  0.014950  0.003524   
question5     0.010693  0.032126  0.031084  0.052004  0.013974  0.011270   
question6     0.007162  0.019703  0.035021  0.036110  0.024863  0.015466   
question7     0.004954  0.020741  0.021978  0.048676  0.027592  0.022431   
question8     0.004826  0.022036  0.044986  0.093683  0.013291  0.017223   
question9     0.003904  0.039761  0.041571  0.032404  0.016084  0.038943   
question10    0.010094  0.029371  0.029922  0.033275  0.011875  0.022299   

                AU09      AU10      AU12      AU14      AU15      AU17  \
label        

            Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                      
question1     0.000398  0.011417  0.008594  0.019907  0.019045  0.005185   
question2     0.000241  0.030874  0.050325  0.012910  0.045342  0.008348   
question3     0.000470  0.016465  0.015409  0.009726  0.023735  0.002979   
question4     0.000220  0.008991  0.008120  0.013249  0.006722  0.005884   
question5     0.000145  0.005244  0.010738  0.029919  0.012149  0.007387   
question6     0.000419  0.005499  0.009252  0.015031  0.005665  0.014133   
question7     0.000297  0.012726  0.019425  0.029936  0.004547  0.001553   
question8     0.000358  0.009862  0.019316  0.018037  0.008518  0.012407   
question9     0.000351  0.021577  0.035694  0.039473  0.011761  0.003182   
question10    0.000950  0.006261  0.015211  0.021069  0.007788  0.009716   

                AU09      AU10      AU12      AU14      AU15      AU17  \
label        

In [3]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

# 削除する列を指定
columns_to_drop = ['Confidence','label', 'AU07', 'AU11', 'AU20']

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # SVMモデルの初期化
    model = SVC(kernel='rbf', probability=True)# 'rbf'カーネルを使用（非線形データの場合）
    model.fit(X_train_combined, y_train_combined)
    
    # 予測
    y_pred = model.predict(X_test)
    y_pred_prob = model.predict_proba(X_test)[:, 1]  # クラス1の確率
    
    # 精度計算
    accuracy = accuracy_score(y_test, y_pred) * 100  # 正答率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")  # 小数点以下2桁で%表示
    print(f"正解ラベル: {y_test.tolist()}")  # 正解ラベルをリストとして表示
    print(f"予測ラベル: {y_pred.tolist()}")  # 予測ラベルをリストとして表示
    print(f"クラス1の確率: {y_pred_prob}")  # 各テストデータ点のクラス1の確率

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")


===== テストデータ: ファイル 1/31 =====
ファイル名: filtered_ID30_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.36333732 0.3981764  0.36256894 0.31804615 0.35576928 0.35088002
 0.36215006 0.44475548 0.3418412  0.35068083]

===== テストデータ: ファイル 2/31 =====
ファイル名: filtered_ID31_smoothed.csv
予測率: 90.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
クラス1の確率: [0.35936444 0.34528683 0.42579791 0.38656269 0.36765839 0.3567892
 0.39455594 0.45491843 0.36350009 0.36630226]

===== テストデータ: ファイル 3/31 =====
ファイル名: filtered_ID32_smoothed.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.3503076  0.27430786 0.25561377 0.27205988 0.31471804 0.27906256
 0.27075497 0.26543099 0.29317668]

===== テストデータ: ファイル 4/31 =====
ファイル名: filtered_ID33_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.30215145 0.33578185 0.33259

In [4]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report

# 削除する列を指定
columns_to_drop = ['Confidence', 'label', 'AU07', 'AU11', 'AU20']

# グリッドサーチ用のパラメータ設定
param_grid = {
    'C': [0.1, 1, 10, 100],         # 正則化パラメータ
    'gamma': [1, 0.1, 0.01, 0.001], # RBFカーネルのgamma
    'kernel': ['rbf']               # カーネルはRBF
}

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # SVMモデルのグリッドサーチ
    model = SVC(probability=True)
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=3, n_jobs=-1)
    grid_search.fit(X_train_combined, y_train_combined)
    
    # 最適なモデルを取得
    best_model = grid_search.best_estimator_
    print(f"最適パラメータ: {grid_search.best_params_}")
    
    # テストデータで予測
    y_pred = best_model.predict(X_test)
    y_pred_prob = best_model.predict_proba(X_test)[:, 1]  # クラス1の確率
    
    # 精度計算
    accuracy = accuracy_score(y_test, y_pred) * 100  # 正答率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")
    print(f"正解ラベル: {y_test.tolist()}")
    print(f"予測ラベル: {y_pred.tolist()}")
    print(f"クラス1の確率: {y_pred_prob}")
    
# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")


===== テストデータ: ファイル 1/31 =====
最適パラメータ: {'C': 0.1, 'gamma': 1, 'kernel': 'rbf'}
ファイル名: filtered_ID30_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.40691016 0.40424111 0.33719252 0.31672119 0.39931935 0.38655027
 0.37616439 0.41963773 0.36279171 0.34304519]

===== テストデータ: ファイル 2/31 =====
最適パラメータ: {'C': 100, 'gamma': 1, 'kernel': 'rbf'}
ファイル名: filtered_ID31_smoothed.csv
予測率: 80.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 1, 0, 0, 0, 0, 1, 0, 0]
クラス1の確率: [0.36291099 0.33092002 0.55212587 0.44202553 0.37356348 0.3353276
 0.37721553 0.52281021 0.45685298 0.39902963]

===== テストデータ: ファイル 3/31 =====
最適パラメータ: {'C': 10, 'gamma': 1, 'kernel': 'rbf'}
ファイル名: filtered_ID32_smoothed.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.36222171 0.2739207  0.30280128 0.31862868 0.30335579 0.3174729
 0.27651763 0.2762825  0.34613986]

===== テストデータ: ファイル 4/31 =====
最適パラメータ: {'C': 0.

最適パラメータ: {'C': 10, 'gamma': 1, 'kernel': 'rbf'}
ファイル名: filtered_ID69_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.36344916 0.38921273 0.36815355 0.34808235 0.40317685 0.36563155
 0.33665096 0.3811223  0.37302118 0.36446937]

===== テストデータ: ファイル 28/31 =====
最適パラメータ: {'C': 0.1, 'gamma': 1, 'kernel': 'rbf'}
ファイル名: filtered_ID70_smoothed.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.38448757 0.40070042 0.38626693 0.30012167 0.40896491 0.33168801
 0.41839046 0.3878711  0.40281171]

===== テストデータ: ファイル 29/31 =====
最適パラメータ: {'C': 10, 'gamma': 1, 'kernel': 'rbf'}
ファイル名: filtered_ID71_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.36725168 0.34278622 0.35972293 0.36071267 0.40497287 0.34984643
 0.36573143 0.3760637  0.36834282 0.35405158]

===== テストデータ: ファイル 30/31 =====
最適パラメータ: {'C': 10, 'gamma': 1, 'kernel': 'r

## SVM（フィルタ後の先頭30フレームの差分）

In [8]:
# 処理するフォルダのパス
folder_path = './output_moving_filtered_data'

# フォルダ内のすべてのCSVファイルを取得
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
labels = [0,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,1,1,1,1,1,0,1,0,0,0]
i = 0
# 各ファイルに対して処理を適用
result_list = []  # 平均値を保存するリスト

for file_name in csv_files:
    file_path = os.path.join(folder_path, file_name)
    
    # データの読み込み
    df = pd.read_csv(file_path, low_memory=False)
    
    # `question1`から`question10`のみをフィルタリング
    filtered_df = df[df["label"].str.match(r'^question(10|[1-9])$', na=False)]
    
    # Confidenceが0.90以上の行を抽出
    filtered_df = filtered_df[filtered_df["Confidence"] >= 0.90]
    
    # `Confidence`列と`AU`で始まる列を抽出
    columns_to_extract = ["label", "Confidence"] + [col for col in filtered_df.columns if col.startswith("AU")]
    filtered_df = filtered_df[columns_to_extract]
    
    # 順序を保つためのカスタム順序のカテゴリを定義
    filtered_df["label"] = pd.Categorical(
        filtered_df["label"],
        categories=[f"question{i}" for i in range(1, 11)],
        ordered=True
    )
    
    # ラベルごとに最初の30フレームを取得
    first_30_frames_per_label = (
        filtered_df.groupby("label")
        .apply(lambda group: group.head(30))
        .reset_index(drop=True)  # グループ化インデックスをリセット
    )
    
    # ラベルごとにフレーム間の差分を計算
    diff_values_per_label = (
        first_30_frames_per_label.groupby("label")
        .apply(lambda group: group.iloc[:, 1:].diff().dropna())
    )

    # 差分に対応するラベル列をリセット（ラベルを戻す）
    diff_values_per_label = diff_values_per_label.reset_index(level=0).dropna()
    
    # 結果をリストに保存
    result_list.append((file_name, diff_values_per_label))

    # Yを追加
    diff_values_per_label["Y"] = labels[i]
    i = i + 1
    
# 各ファイルの結果を出力
for file_name, mean_values in result_list:
    print(f"Results for {file_name}:")
    print(mean_values)
    print()

Results for filtered_ID30_smoothed.csv:
          label  Confidence      AU01      AU02      AU04      AU05      AU06  \
1     question1   -0.001724 -0.002031 -0.009685 -0.039566 -0.007552  0.002423   
2     question1    0.000788  0.004552  0.014109 -0.005977  0.017838  0.002006   
3     question1   -0.000408 -0.001043 -0.000382 -0.001480 -0.008086 -0.010739   
4     question1    0.000278 -0.014017  0.013837 -0.005446 -0.002008 -0.011532   
5     question1    0.001025 -0.010964  0.003417 -0.033513 -0.000709  0.002095   
..          ...         ...       ...       ...       ...       ...       ...   
295  question10   -0.000181 -0.010951 -0.009110 -0.025020 -0.005472  0.000085   
296  question10   -0.001860  0.000080 -0.019191  0.029150 -0.005756  0.004851   
297  question10    0.002063  0.002683 -0.009171  0.038275  0.014208 -0.010177   
298  question10   -0.003481  0.001530 -0.003313  0.003233  0.022098 -0.007301   
299  question10   -0.001027  0.009657  0.012584 -0.008357  0.011535 -

          label  Confidence      AU01      AU02      AU04      AU05      AU06  \
1     question1    0.037247 -0.002903  0.010812 -0.019731  0.001753 -0.007391   
2     question1    0.004702 -0.006709 -0.005719  0.026798 -0.004354  0.005827   
3     question1   -0.041508  0.003873 -0.014272  0.015316 -0.016713  0.007079   
4     question1    0.013944  0.019152  0.013913 -0.009422 -0.015214 -0.006678   
5     question1   -0.004027  0.017828  0.011002 -0.017050 -0.026142 -0.005049   
..          ...         ...       ...       ...       ...       ...       ...   
295  question10   -0.002277  0.006156  0.014720 -0.014244  0.012747 -0.001500   
296  question10   -0.010453  0.001171 -0.002745 -0.029892 -0.002457  0.007685   
297  question10   -0.066103 -0.025797 -0.009471 -0.022133 -0.008375 -0.005177   
298  question10    0.082290 -0.001290 -0.005430  0.010301 -0.015662  0.005689   
299  question10   -0.011013 -0.019531 -0.008989 -0.012441  0.016403  0.009308   

         AU09      AU10    

In [9]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

# 削除する列を指定
columns_to_drop = ['Confidence','label', 'AU07', 'AU11', 'AU20']

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # SVMモデルの初期化
    model = SVC(kernel='rbf', probability=True)# 'rbf'カーネルを使用（非線形データの場合）
    model.fit(X_train_combined, y_train_combined)
    
    # 予測
    y_pred = model.predict(X_test)
    y_pred_prob = model.predict_proba(X_test)[:, 1]  # クラス1の確率
    
    # 精度計算
    accuracy = accuracy_score(y_test, y_pred) * 100  # 正答率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")  # 小数点以下2桁で%表示
    print(f"正解ラベル: {y_test.tolist()}")  # 正解ラベルをリストとして表示
    print(f"予測ラベル: {y_pred.tolist()}")  # 予測ラベルをリストとして表示
    print(f"クラス1の確率: {y_pred_prob}")  # 各テストデータ点のクラス1の確率

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")


===== テストデータ: ファイル 1/31 =====
ファイル名: filtered_ID30_smoothed.csv
予測率: 94.14%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

ファイル名: filtered_ID32_smoothed.csv
予測率: 1.92%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

ファイル名: filtered_ID34_smoothed.csv
予測率: 26.29%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0,

ファイル名: filtered_ID36_smoothed.csv
予測率: 92.76%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,

ファイル名: filtered_ID38_smoothed.csv
予測率: 90.42%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,

...

ファイル名: filtered_ID56_smoothed.csv
予測率: 22.66%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1,

ファイル名: filtered_ID63_smoothed.csv
予測率: 3.07%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

ファイル名: filtered_ID69_smoothed.csv
予測率: 88.62%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,

ファイル名: filtered_ID71_smoothed.csv
予測率: 90.69%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

ファイル名: filtered_ID73_smoothed.csv
予測率: 84.14%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

## SVM（フィルタ後の先頭30フレームの差分の標準偏差）

In [8]:
# 処理するフォルダのパス
folder_path = './output_moving_filtered_data'

# フォルダ内のすべてのCSVファイルを取得
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
labels = [0,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,1,1,1,1,1,0,1,0,0,0]
i = 0
# 各ファイルに対して処理を適用
result_list = []  # 平均値を保存するリスト

for file_name in csv_files:
    file_path = os.path.join(folder_path, file_name)
    
    # データの読み込み
    df = pd.read_csv(file_path, low_memory=False)
    
    # `question1`から`question10`のみをフィルタリング
    filtered_df = df[df["label"].str.match(r'^question(10|[1-9])$', na=False)]
    
    # Confidenceが0.90以上の行を抽出
    filtered_df = filtered_df[filtered_df["Confidence"] >= 0.90]
    
    # `Confidence`列と`AU`で始まる列を抽出
    columns_to_extract = ["label", "Confidence"] + [col for col in filtered_df.columns if col.startswith("AU")]
    filtered_df = filtered_df[columns_to_extract]
    
    # 順序を保つためのカスタム順序のカテゴリを定義
    filtered_df["label"] = pd.Categorical(
        filtered_df["label"],
        categories=[f"question{i}" for i in range(1, 11)],
        ordered=True
    )
    
    # ラベルごとに最初の30フレームを取得
    first_30_frames_per_label = (
        filtered_df.groupby("label")
        .apply(lambda group: group.head(30))
        .reset_index(drop=True)  # グループ化インデックスをリセット
    )
    
    # ラベルごとにフレーム間の差分を計算
    diff_values_per_label = (
        first_30_frames_per_label.groupby("label")
        .apply(lambda group: group.iloc[:, 1:].diff().dropna())
    )

    # 差分に対応するラベル列をリセット（ラベルを戻す）
    diff_values_per_label = diff_values_per_label.reset_index(level=0).dropna()
    
    # 差分の標準偏差を計算
    std_values_per_label = (
        diff_values_per_label.groupby("label").std(numeric_only=True)
    )
    
    # 結果をリストに保存
    result_list.append((file_name, std_values_per_label))

    # Yを追加
    std_values_per_label["Y"] = labels[i]
    i = i + 1
    
# 各ファイルの結果を出力
for file_name, mean_values in result_list:
    print(f"Results for {file_name}:")
    print(mean_values)
    print()

Results for filtered_ID30_smoothed.csv:
            Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                      
question1     0.001357  0.011945  0.011015  0.017625  0.008104  0.005981   
question2     0.001140  0.009808  0.013247  0.013333  0.009484  0.007334   
question3     0.001540  0.011213  0.013871  0.017764  0.010590  0.004507   
question4     0.001450  0.011179  0.011102  0.015170  0.010495  0.005445   
question5     0.001605  0.009229  0.013612  0.023189  0.007825  0.013743   
question6     0.001716  0.007972  0.011852  0.015273  0.010305  0.008880   
question7     0.002743  0.010369  0.011425  0.022203  0.015390  0.004805   
question8     0.001929  0.010254  0.017442  0.022008  0.004848  0.010167   
question9     0.001081  0.011445  0.013607  0.015146  0.010546  0.005656   
question10    0.001231  0.008036  0.010590  0.025266  0.011607  0.004519   

                AU09      AU10      AU12      A

            Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                      
question1     0.000479  0.010356  0.015096  0.014286  0.007540  0.007224   
question2     0.000019  0.003267  0.009640  0.015676  0.006550  0.006649   
question3     0.000019  0.005147  0.005465  0.011564  0.009127  0.003586   
question4     0.000017  0.006166  0.009504  0.014390  0.013766  0.004254   
question5     0.000017  0.007283  0.009518  0.011021  0.001656  0.005599   
question6     0.000037  0.007722  0.019344  0.011228  0.001776  0.004727   
question7     0.000013  0.005190  0.011578  0.014485  0.008661  0.010035   
question8     0.000144  0.013194  0.015564  0.019242  0.003799  0.016772   
question9     0.000029  0.010358  0.009874  0.013481  0.004117  0.008097   
question10    0.000128  0.007997  0.014553  0.016975  0.007916  0.013497   

                AU09      AU10      AU12      AU14      AU15      AU17  \
label        

In [11]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

# 削除する列を指定
columns_to_drop = ['Confidence','label', 'AU07', 'AU11', 'AU20']

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # SVMモデルの初期化
    model = SVC(kernel='rbf', probability=True)# 'rbf'カーネルを使用（非線形データの場合）
    model.fit(X_train_combined, y_train_combined)
    
    # 予測
    y_pred = model.predict(X_test)
    y_pred_prob = model.predict_proba(X_test)[:, 1]  # クラス1の確率
    
    # 精度計算
    accuracy = accuracy_score(y_test, y_pred) * 100  # 正答率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")  # 小数点以下2桁で%表示
    print(f"正解ラベル: {y_test.tolist()}")  # 正解ラベルをリストとして表示
    print(f"予測ラベル: {y_pred.tolist()}")  # 予測ラベルをリストとして表示
    print(f"クラス1の確率: {y_pred_prob}")  # 各テストデータ点のクラス1の確率

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")


===== テストデータ: ファイル 1/31 =====
ファイル名: filtered_ID30_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.24700116 0.24858523 0.28622543 0.2571501  0.22772704 0.18299183
 0.31946606 0.24118342 0.20698033 0.40647693]

===== テストデータ: ファイル 2/31 =====
ファイル名: filtered_ID31_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.2593756  0.13592809 0.2849456  0.27806381 0.20480275 0.3542843
 0.22470511 0.35992124 0.23719054 0.21731562]

===== テストデータ: ファイル 3/31 =====
ファイル名: filtered_ID32_smoothed.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.14356654 0.0961847  0.12723363 0.11738009 0.14285138 0.13879722
 0.14564187 0.11665404 0.14430686]

===== テストデータ: ファイル 4/31 =====
ファイル名: filtered_ID33_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.20011512 0.32766537 0.2912

In [9]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report

# 削除する列を指定
columns_to_drop = ['Confidence', 'label', 'AU07', 'AU11', 'AU20']

# グリッドサーチ用のパラメータ設定
param_grid = {
    'C': [0.1, 1, 10, 100],         # 正則化パラメータ
    'gamma': [1, 0.1, 0.01, 0.001], # RBFカーネルのgamma
    'kernel': ['rbf']               # カーネルはRBF
}

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # SVMモデルのグリッドサーチ
    model = SVC(probability=True)
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=3, n_jobs=-1)
    grid_search.fit(X_train_combined, y_train_combined)
    
    # 最適なモデルを取得
    best_model = grid_search.best_estimator_
    print(f"最適パラメータ: {grid_search.best_params_}")
    
    # テストデータで予測
    y_pred = best_model.predict(X_test)
    y_pred_prob = best_model.predict_proba(X_test)[:, 1]  # クラス1の確率
    
    # 精度計算
    accuracy = accuracy_score(y_test, y_pred) * 100  # 正答率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")
    print(f"正解ラベル: {y_test.tolist()}")
    print(f"予測ラベル: {y_pred.tolist()}")
    print(f"クラス1の確率: {y_pred_prob}")
    
# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")



===== テストデータ: ファイル 1/31 =====
最適パラメータ: {'C': 100, 'gamma': 1, 'kernel': 'rbf'}
ファイル名: filtered_ID30_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.34907551 0.31726589 0.3623106  0.30418617 0.31484434 0.26930587
 0.35468657 0.36769299 0.29672776 0.38481518]

===== テストデータ: ファイル 2/31 =====
最適パラメータ: {'C': 100, 'gamma': 1, 'kernel': 'rbf'}
ファイル名: filtered_ID31_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.31472452 0.24798998 0.23389717 0.26073482 0.22459958 0.33653773
 0.25409343 0.49311756 0.29852789 0.23604986]

===== テストデータ: ファイル 3/31 =====
最適パラメータ: {'C': 10, 'gamma': 1, 'kernel': 'rbf'}
ファイル名: filtered_ID32_smoothed.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.30322181 0.19179627 0.13575954 0.15026908 0.31786716 0.17035759
 0.16590585 0.15462861 0.24523697]

===== テストデータ: ファイル 4/31 =====
最適パラメータ: {'C':

最適パラメータ: {'C': 10, 'gamma': 1, 'kernel': 'rbf'}
ファイル名: filtered_ID69_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.43633443 0.2675815  0.40342478 0.27992917 0.40106259 0.25517793
 0.32261045 0.31103351 0.41171142 0.32886374]

===== テストデータ: ファイル 28/31 =====
最適パラメータ: {'C': 0.1, 'gamma': 1, 'kernel': 'rbf'}
ファイル名: filtered_ID70_smoothed.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.7987912  0.85032786 0.63479842 0.84687099 0.64139923 0.52469951
 0.77251289 0.79293129 0.74565037]

===== テストデータ: ファイル 29/31 =====
最適パラメータ: {'C': 10, 'gamma': 1, 'kernel': 'rbf'}
ファイル名: filtered_ID71_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.37523908 0.22174235 0.34208833 0.3786631  0.39670853 0.20349899
 0.20278241 0.19751625 0.34446211 0.17898654]

===== テストデータ: ファイル 30/31 =====
最適パラメータ: {'C': 10, 'gamma': 1, 'kernel': 'r

## SVM（フィルタ後の先頭30フレームの最大値）

In [10]:
# 処理するフォルダのパス
folder_path = './output_moving_filtered_data'

# フォルダ内のすべてのCSVファイルを取得
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
labels = [0,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,1,1,1,1,1,0,1,0,0,0]
i = 0
# 各ファイルに対して処理を適用
result_list = []  # 平均値を保存するリスト

for file_name in csv_files:
    file_path = os.path.join(folder_path, file_name)
    
    # データの読み込み
    df = pd.read_csv(file_path, low_memory=False)
    
    # `question1`から`question10`のみをフィルタリング
    filtered_df = df[df["label"].str.match(r'^question(10|[1-9])$', na=False)]
    
    # Confidenceが0.90以上の行を抽出
    filtered_df = filtered_df[filtered_df["Confidence"] >= 0.90]
    
    # `Confidence`列と`AU`で始まる列を抽出
    columns_to_extract = ["label", "Confidence"] + [col for col in filtered_df.columns if col.startswith("AU")]
    filtered_df = filtered_df[columns_to_extract]
    
    # 順序を保つためのカスタム順序のカテゴリを定義
    filtered_df["label"] = pd.Categorical(
        filtered_df["label"],
        categories=[f"question{i}" for i in range(1, 11)],
        ordered=True
    )
    
    # ラベルごとに最初の30フレームを取得
    first_30_frames_per_label = (
        filtered_df.groupby("label")
        .apply(lambda group: group.head(30))
        .reset_index(drop=True)  # グループ化インデックスをリセット
    )

    # ラベルごとに30フレームの最大値を計算
    max_values_per_label = (
        first_30_frames_per_label.groupby("label").max(numeric_only=True)
    )
    
    # 結果をリストに保存
    result_list.append((file_name, max_values_per_label))

    # Yを追加
    max_values_per_label["Y"] = labels[i]
    i = i + 1
    
# 各ファイルの結果を出力
for file_name, mean_values in result_list:
    print(f"Results for {file_name}:")
    print(mean_values)
    print()

Results for filtered_ID30_smoothed.csv:
            Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                      
question1     0.998051  0.629318  0.309042  0.560439  0.350857  0.226028   
question2     0.998089  0.709137  0.531324  0.592470  0.349076  0.210350   
question3     0.998311  0.679692  0.416384  0.495837  0.358993  0.189606   
question4     0.998152  0.653097  0.327153  0.503030  0.373655  0.200907   
question5     0.997102  0.648369  0.333211  0.637200  0.354853  0.362731   
question6     0.998499  0.672398  0.391179  0.546530  0.479017  0.273631   
question7     0.997608  0.690041  0.391348  0.550019  0.489479  0.240377   
question8     0.998259  0.596494  0.398190  0.536899  0.332791  0.330788   
question9     0.998850  0.773111  0.433624  0.547648  0.425296  0.241967   
question10    0.998399  0.773570  0.443391  0.571621  0.455789  0.209519   

                AU09      AU10      AU12      A

            Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                      
question1     0.999527  0.786934  0.554521  0.384096  0.431120  0.190257   
question2     0.999877  0.815830  0.588060  0.475021  0.444084  0.241001   
question3     0.999848  0.771271  0.495804  0.281035  0.403756  0.174407   
question4     0.999871  0.714677  0.462773  0.376953  0.414113  0.201549   
question5     0.999870  0.748691  0.510393  0.401699  0.261175  0.273601   
question6     0.999795  0.724730  0.473838  0.442669  0.267487  0.273642   
question7     0.999896  0.854910  0.633331  0.284574  0.375448  0.401482   
question8     0.999919  0.803451  0.575835  0.663083  0.293925  0.547306   
question9     0.999830  0.740983  0.451885  0.431989  0.291078  0.254937   
question10    0.999820  0.694747  0.540105  0.451897  0.395379  0.360120   

                AU09      AU10      AU12      AU14      AU15      AU17  \
label        

            Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                      
question1          NaN       NaN       NaN       NaN       NaN       NaN   
question2     0.982307  0.626160  0.518810  0.620210  0.369059  0.243927   
question3          NaN       NaN       NaN       NaN       NaN       NaN   
question4     0.994893  0.809002  0.715695  0.506057  0.301297  0.076125   
question5     0.996796  0.771329  0.613417  0.401279  0.299089  0.148697   
question6     0.995173  0.581545  0.432393  0.513171  0.301157  0.183523   
question7          NaN       NaN       NaN       NaN       NaN       NaN   
question8     0.996467  0.709256  0.591232  0.491880  0.285602  0.133768   
question9     0.995671  0.777240  0.637825  0.431719  0.283662  0.165330   
question10    0.991356  0.700055  0.551304  0.512816  0.309428  0.224100   

                AU09      AU10      AU12      AU14      AU15      AU17  \
label        

In [13]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

# 削除する列を指定
columns_to_drop = ['Confidence','label', 'AU07', 'AU11', 'AU20']

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # SVMモデルの初期化
    model = SVC(kernel='rbf', probability=True)# 'rbf'カーネルを使用（非線形データの場合）
    model.fit(X_train_combined, y_train_combined)
    
    # 予測
    y_pred = model.predict(X_test)
    y_pred_prob = model.predict_proba(X_test)[:, 1]  # クラス1の確率
    
    # 精度計算
    accuracy = accuracy_score(y_test, y_pred) * 100  # 正答率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")  # 小数点以下2桁で%表示
    print(f"正解ラベル: {y_test.tolist()}")  # 正解ラベルをリストとして表示
    print(f"予測ラベル: {y_pred.tolist()}")  # 予測ラベルをリストとして表示
    print(f"クラス1の確率: {y_pred_prob}")  # 各テストデータ点のクラス1の確率

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")


===== テストデータ: ファイル 1/31 =====
ファイル名: filtered_ID30_smoothed.csv
予測率: 70.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 1, 0, 0, 0, 1, 1]
クラス1の確率: [0.308939   0.5        0.38245533 0.22673664 0.59751359 0.5
 0.52191498 0.51546191 0.57927436 0.56349335]

===== テストデータ: ファイル 2/31 =====
ファイル名: filtered_ID31_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.06550831 0.03786092 0.06885591 0.0724723  0.06366714 0.16522049
 0.06993966 0.0959548  0.04583173 0.03275101]

===== テストデータ: ファイル 3/31 =====
ファイル名: filtered_ID32_smoothed.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.02639064 0.05762892 0.09823772 0.11618246 0.13471085 0.05637637
 0.0930747  0.15615987 0.01918203]

===== テストデータ: ファイル 4/31 =====
ファイル名: filtered_ID33_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.05029538 0.21076882 0.26127459 0.

In [11]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report

# 削除する列を指定
columns_to_drop = ['Confidence', 'label', 'AU07', 'AU11', 'AU20']

# グリッドサーチ用のパラメータ設定
param_grid = {
    'C': [0.1, 1, 10, 100],         # 正則化パラメータ
    'gamma': [1, 0.1, 0.01, 0.001], # RBFカーネルのgamma
    'kernel': ['rbf']               # カーネルはRBF
}

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # SVMモデルのグリッドサーチ
    model = SVC(probability=True)
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=3, n_jobs=-1)
    grid_search.fit(X_train_combined, y_train_combined)
    
    # 最適なモデルを取得
    best_model = grid_search.best_estimator_
    print(f"最適パラメータ: {grid_search.best_params_}")
    
    # テストデータで予測
    y_pred = best_model.predict(X_test)
    y_pred_prob = best_model.predict_proba(X_test)[:, 1]  # クラス1の確率
    
    # 精度計算
    accuracy = accuracy_score(y_test, y_pred) * 100  # 正答率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")
    print(f"正解ラベル: {y_test.tolist()}")
    print(f"予測ラベル: {y_pred.tolist()}")
    print(f"クラス1の確率: {y_pred_prob}")
    
# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")



===== テストデータ: ファイル 1/31 =====
最適パラメータ: {'C': 100, 'gamma': 0.01, 'kernel': 'rbf'}
ファイル名: filtered_ID30_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.31490364 0.44480599 0.32780307 0.20847687 0.5        0.39492326
 0.46429128 0.43521371 0.45361489 0.48053901]

===== テストデータ: ファイル 2/31 =====
最適パラメータ: {'C': 100, 'gamma': 0.01, 'kernel': 'rbf'}
ファイル名: filtered_ID31_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.11913184 0.05404114 0.03140492 0.11479447 0.10955496 0.20782923
 0.10681337 0.19470345 0.07741089 0.06682118]

===== テストデータ: ファイル 3/31 =====
最適パラメータ: {'C': 100, 'gamma': 0.01, 'kernel': 'rbf'}
ファイル名: filtered_ID32_smoothed.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.13645003 0.13894513 0.14735104 0.15596213 0.17106968 0.12935598
 0.1306141  0.19203768 0.06193932]

===== テストデータ: ファイル 4/31 =====
最適パラ

最適パラメータ: {'C': 0.1, 'gamma': 1, 'kernel': 'rbf'}
ファイル名: filtered_ID69_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.70596846 0.31248921 0.50981892 0.27516782 0.11457235 0.4350975
 0.10447893 0.54248486 0.14122105 0.21664677]

===== テストデータ: ファイル 28/31 =====
最適パラメータ: {'C': 100, 'gamma': 0.01, 'kernel': 'rbf'}
ファイル名: filtered_ID70_smoothed.csv
予測率: 55.56%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 1, 0, 0, 0, 1, 1, 1, 1]
クラス1の確率: [0.31026045 0.6346982  0.48585775 0.36150861 0.19950005 0.65259545
 0.69543451 0.76218887 0.76456737]

===== テストデータ: ファイル 29/31 =====
最適パラメータ: {'C': 100, 'gamma': 0.1, 'kernel': 'rbf'}
ファイル名: filtered_ID71_smoothed.csv
予測率: 50.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [1, 0, 0, 0, 1, 0, 1, 1, 1, 0]
クラス1の確率: [0.72242151 0.25662121 0.33338135 0.33581892 0.55354717 0.40836338
 0.49009458 0.46375962 0.51361895 0.17301353]

===== テストデータ: ファイル 30/31 =====
最適パラメータ: {'C': 100, 'gamma': 0.01, 'k

## SVM（フィルタ後の先頭10フレームの最大値）

In [12]:
# 処理するフォルダのパス
folder_path = './output_moving_filtered_data'

# フォルダ内のすべてのCSVファイルを取得
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
labels = [0,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,1,1,1,1,1,0,1,0,0,0]
i = 0
# 各ファイルに対して処理を適用
result_list = []  # 平均値を保存するリスト

for file_name in csv_files:
    file_path = os.path.join(folder_path, file_name)
    
    # データの読み込み
    df = pd.read_csv(file_path, low_memory=False)
    
    # `question1`から`question10`のみをフィルタリング
    filtered_df = df[df["label"].str.match(r'^question(10|[1-9])$', na=False)]
    
    # Confidenceが0.90以上の行を抽出
    filtered_df = filtered_df[filtered_df["Confidence"] >= 0.90]
    
    # `Confidence`列と`AU`で始まる列を抽出
    columns_to_extract = ["label", "Confidence"] + [col for col in filtered_df.columns if col.startswith("AU")]
    filtered_df = filtered_df[columns_to_extract]
    
    # 順序を保つためのカスタム順序のカテゴリを定義
    filtered_df["label"] = pd.Categorical(
        filtered_df["label"],
        categories=[f"question{i}" for i in range(1, 11)],
        ordered=True
    )
    
    # ラベルごとに最初の10フレームを取得
    first_10_frames_per_label = (
        filtered_df.groupby("label")
        .apply(lambda group: group.head(10))
        .reset_index(drop=True)  # グループ化インデックスをリセット
    )

    # ラベルごとに10フレームの最大値を計算
    max_values_per_label = (
        first_10_frames_per_label.groupby("label").max(numeric_only=True)
    )
    
    # 結果をリストに保存
    result_list.append((file_name, max_values_per_label))

    # Yを追加
    max_values_per_label["Y"] = labels[i]
    i = i + 1
    
# 各ファイルの結果を出力
for file_name, mean_values in result_list:
    print(f"Results for {file_name}:")
    print(mean_values)
    print()

Results for filtered_ID30_smoothed.csv:
            Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                      
question1     0.996113  0.574940  0.309042  0.455083  0.350857  0.226028   
question2     0.996619  0.709137  0.531324  0.592470  0.301983  0.210350   
question3     0.998311  0.666825  0.361777  0.495837  0.347845  0.174839   
question4     0.997968  0.619893  0.321957  0.503030  0.348712  0.200907   
question5     0.997069  0.606037  0.327818  0.637200  0.342222  0.359800   
question6     0.998499  0.620801  0.391179  0.546530  0.458388  0.273631   
question7     0.996357  0.690041  0.391348  0.550019  0.436465  0.240377   
question8     0.998259  0.577708  0.339130  0.536899  0.330281  0.277568   
question9     0.998850  0.727150  0.433624  0.488248  0.425296  0.239437   
question10    0.998399  0.731874  0.439494  0.484566  0.455789  0.202454   

                AU09      AU10      AU12      A

            Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                      
question1     0.998606  0.725770  0.451675  0.384096  0.312047  0.190257   
question2     0.999836  0.815830  0.535682  0.475021  0.420100  0.241001   
question3     0.999835  0.771271  0.495804  0.208797  0.339738  0.145478   
question4     0.999867  0.713594  0.462773  0.376953  0.403494  0.181692   
question5     0.999870  0.742508  0.504851  0.297880  0.230461  0.273601   
question6     0.999772  0.724730  0.459851  0.442669  0.267487  0.247753   
question7     0.999893  0.849498  0.579854  0.284574  0.314148  0.306524   
question8     0.999628  0.473799  0.543951  0.663083  0.281739  0.547306   
question9     0.999809  0.709568  0.423699  0.431989  0.291078  0.250853   
question10    0.999569  0.683709  0.437678  0.451897  0.385885  0.353783   

                AU09      AU10      AU12      AU14      AU15      AU17  \
label        

            Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                      
question1     0.999052  0.432012  0.234709  0.664503  0.390222  0.214668   
question2     0.998651  0.475051  0.296463  0.670512  0.334677  0.290464   
question3     0.999080  0.449310  0.326970  0.677686  0.387250  0.080217   
question4     0.999030  0.515099  0.443431  0.700717  0.337915  0.097634   
question5     0.998931  0.334304  0.256269  0.818416  0.336176  0.170953   
question6     0.998190  0.367121  0.224968  0.763646  0.307444  0.169341   
question7     0.998412  0.420278  0.281416  0.655659  0.363186  0.145052   
question8     0.998579  0.560053  0.483984  0.539264  0.357977  0.119231   
question9     0.998787  0.509015  0.475774  0.473312  0.364162  0.129932   
question10    0.998432  0.602467  0.513427  0.539503  0.379681  0.085721   

                AU09      AU10      AU12      AU14      AU15      AU17  \
label        

In [15]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

# 削除する列を指定
columns_to_drop = ['Confidence','label', 'AU07', 'AU11', 'AU20']

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # SVMモデルの初期化
    model = SVC(kernel='rbf', probability=True)# 'rbf'カーネルを使用（非線形データの場合）
    model.fit(X_train_combined, y_train_combined)
    
    # 予測
    y_pred = model.predict(X_test)
    y_pred_prob = model.predict_proba(X_test)[:, 1]  # クラス1の確率
    
    # 精度計算
    accuracy = accuracy_score(y_test, y_pred) * 100  # 正答率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")  # 小数点以下2桁で%表示
    print(f"正解ラベル: {y_test.tolist()}")  # 正解ラベルをリストとして表示
    print(f"予測ラベル: {y_pred.tolist()}")  # 予測ラベルをリストとして表示
    print(f"クラス1の確率: {y_pred_prob}")  # 各テストデータ点のクラス1の確率

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")


===== テストデータ: ファイル 1/31 =====
ファイル名: filtered_ID30_smoothed.csv
予測率: 50.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 1, 0, 0, 1, 1, 0, 0, 1, 1]
クラス1の確率: [0.42703989 0.67233889 0.32315796 0.21831813 0.57975624 0.57856464
 0.5236839  0.46115883 0.66734415 0.69817548]

===== テストデータ: ファイル 2/31 =====
ファイル名: filtered_ID31_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.14798084 0.08565045 0.10745149 0.11314532 0.06381564 0.08174937
 0.09209776 0.25855224 0.0820375  0.0439044 ]

===== テストデータ: ファイル 3/31 =====
ファイル名: filtered_ID32_smoothed.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.06078334 0.07113873 0.05787679 0.06193675 0.08245716 0.03838849
 0.11639574 0.1398435  0.01620656]

===== テストデータ: ファイル 4/31 =====
ファイル名: filtered_ID33_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.19567877 0.27749406 0.3121

In [13]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report

# 削除する列を指定
columns_to_drop = ['Confidence', 'label', 'AU07', 'AU11', 'AU20']

# グリッドサーチ用のパラメータ設定
param_grid = {
    'C': [0.1, 1, 10, 100],         # 正則化パラメータ
    'gamma': [1, 0.1, 0.01, 0.001], # RBFカーネルのgamma
    'kernel': ['rbf']               # カーネルはRBF
}

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # SVMモデルのグリッドサーチ
    model = SVC(probability=True)
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=3, n_jobs=-1)
    grid_search.fit(X_train_combined, y_train_combined)
    
    # 最適なモデルを取得
    best_model = grid_search.best_estimator_
    print(f"最適パラメータ: {grid_search.best_params_}")
    
    # テストデータで予測
    y_pred = best_model.predict(X_test)
    y_pred_prob = best_model.predict_proba(X_test)[:, 1]  # クラス1の確率
    
    # 精度計算
    accuracy = accuracy_score(y_test, y_pred) * 100  # 正答率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")
    print(f"正解ラベル: {y_test.tolist()}")
    print(f"予測ラベル: {y_pred.tolist()}")
    print(f"クラス1の確率: {y_pred_prob}")
    
# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")



===== テストデータ: ファイル 1/31 =====
最適パラメータ: {'C': 100, 'gamma': 0.01, 'kernel': 'rbf'}
ファイル名: filtered_ID30_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.35906932 0.3999985  0.24207486 0.195523   0.44887383 0.3698975
 0.40191136 0.38541135 0.46432389 0.46843445]

===== テストデータ: ファイル 2/31 =====
最適パラメータ: {'C': 100, 'gamma': 0.01, 'kernel': 'rbf'}
ファイル名: filtered_ID31_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.14944619 0.08643732 0.04319833 0.15499686 0.11823829 0.1105788
 0.11530626 0.21447343 0.11655364 0.08318894]

===== テストデータ: ファイル 3/31 =====
最適パラメータ: {'C': 100, 'gamma': 0.01, 'kernel': 'rbf'}
ファイル名: filtered_ID32_smoothed.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.17361378 0.10931592 0.10329087 0.10695066 0.13275465 0.10488397
 0.13333257 0.1487599  0.04092665]

===== テストデータ: ファイル 4/31 =====
最適パラメー

最適パラメータ: {'C': 100, 'gamma': 0.01, 'kernel': 'rbf'}
ファイル名: filtered_ID69_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.51988453 0.4790224  0.45163621 0.27747773 0.13547517 0.43875794
 0.09232773 0.41936281 0.16332406 0.35793284]

===== テストデータ: ファイル 28/31 =====
最適パラメータ: {'C': 100, 'gamma': 0.01, 'kernel': 'rbf'}
ファイル名: filtered_ID70_smoothed.csv
予測率: 33.33%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 1, 0, 1, 1]
クラス1の確率: [0.27432067 0.50624801 0.40568369 0.21060312 0.10816389 0.64433656
 0.52520941 0.68360584 0.66257594]

===== テストデータ: ファイル 29/31 =====
最適パラメータ: {'C': 100, 'gamma': 0.01, 'kernel': 'rbf'}
ファイル名: filtered_ID71_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.40309772 0.22850128 0.1884233  0.31874503 0.34050144 0.15885344
 0.27188206 0.24998621 0.22644813 0.16993575]

===== テストデータ: ファイル 30/31 =====
最適パラメータ: {'C': 100, 'gamma': 0.

## SVM（フィルタ後の先頭30フレームの最大値と最小値の差分）

In [14]:
# 処理するフォルダのパス
folder_path = './output_moving_filtered_data'

# フォルダ内のすべてのCSVファイルを取得
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
labels = [0,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,1,1,1,1,1,0,1,0,0,0]
i = 0
# 各ファイルに対して処理を適用
result_list = []  # 平均値を保存するリスト

for file_name in csv_files:
    file_path = os.path.join(folder_path, file_name)
    
    # データの読み込み
    df = pd.read_csv(file_path, low_memory=False)
    
    # `question1`から`question10`のみをフィルタリング
    filtered_df = df[df["label"].str.match(r'^question(10|[1-9])$', na=False)]
    
    # Confidenceが0.90以上の行を抽出
    filtered_df = filtered_df[filtered_df["Confidence"] >= 0.90]
    
    # `Confidence`列と`AU`で始まる列を抽出
    columns_to_extract = ["label", "Confidence"] + [col for col in filtered_df.columns if col.startswith("AU")]
    filtered_df = filtered_df[columns_to_extract]
    
    # 順序を保つためのカスタム順序のカテゴリを定義
    filtered_df["label"] = pd.Categorical(
        filtered_df["label"],
        categories=[f"question{i}" for i in range(1, 11)],
        ordered=True
    )
    
    # ラベルごとに最初の30フレームを取得
    first_30_frames_per_label = (
        filtered_df.groupby("label")
        .apply(lambda group: group.head(30))
        .reset_index(drop=True)  # グループ化インデックスをリセット
    )

    # ラベルごとに最大値を計算
    max_values_per_label = (
        first_30_frames_per_label.groupby("label").max(numeric_only=True)
    )

    # ラベルごとに最小値を計算
    min_values_per_label = (
        first_30_frames_per_label.groupby("label").min(numeric_only=True)
    )

    # 最大値と最小値の差分を計算
    range_values_per_label = max_values_per_label - min_values_per_label

    
    # 結果をリストに保存
    result_list.append((file_name, range_values_per_label))

    # Yを追加
    range_values_per_label["Y"] = labels[i]
    i = i + 1
    
# 各ファイルの結果を出力
for file_name, mean_values in result_list:
    print(f"Results for {file_name}:")
    print(mean_values)
    print()

Results for filtered_ID30_smoothed.csv:
            Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                      
question1     0.005760  0.091032  0.089920  0.191338  0.080192  0.074836   
question2     0.005446  0.148309  0.230588  0.127627  0.091902  0.060158   
question3     0.007495  0.063130  0.116779  0.076798  0.059728  0.049990   
question4     0.006056  0.085221  0.054412  0.130832  0.095369  0.034621   
question5     0.005688  0.059413  0.127185  0.122237  0.056385  0.113033   
question6     0.008136  0.088952  0.054877  0.111280  0.078245  0.071187   
question7     0.009249  0.099774  0.071603  0.178759  0.139358  0.056048   
question8     0.005283  0.043323  0.144713  0.144408  0.031742  0.069094   
question9     0.004680  0.101396  0.100224  0.097717  0.107856  0.035054   
question10    0.005946  0.062949  0.066295  0.180012  0.138571  0.024668   

                AU09      AU10      AU12      A

            Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                      
question1          NaN       NaN       NaN       NaN       NaN       NaN   
question2     0.000090  0.126324  0.097394  0.059522  0.090151  0.031222   
question3     0.000098  0.047188  0.227359  0.201140  0.037565  0.131017   
question4     0.000046  0.076812  0.061728  0.145558  0.163916  0.041110   
question5     0.000051  0.131085  0.126695  0.217849  0.178862  0.071674   
question6     0.000110  0.141646  0.058703  0.191294  0.100916  0.059133   
question7     0.000744  0.113057  0.077428  0.097956  0.063184  0.091156   
question8     0.000173  0.046278  0.081878  0.135486  0.089993  0.030588   
question9     0.000080  0.147696  0.117281  0.200992  0.132054  0.109615   
question10    0.000100  0.063516  0.027307  0.073513  0.060277  0.070664   

                AU09      AU10      AU12      AU14      AU15      AU17  \
label        

            Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                      
question1     0.002133  0.255925  0.284608  0.198036  0.122238  0.149684   
question2     0.002941  0.096096  0.092117  0.193493  0.216071  0.030609   
question3     0.007515  0.110485  0.195642  0.234579  0.147745  0.178324   
question4     0.018290  0.114110  0.160241  0.165183  0.162667  0.125246   
question5     0.009931  0.195570  0.121268  0.122025  0.082890  0.058395   
question6     0.012080  0.140727  0.146211  0.144667  0.094683  0.077550   
question7     0.007074  0.105297  0.177336  0.168991  0.066687  0.074888   
question8     0.020769  0.138386  0.324653  0.334909  0.129705  0.109727   
question9     0.022698  0.095998  0.170592  0.103688  0.060799  0.094507   
question10    0.014564  0.074358  0.260750  0.258180  0.086683  0.121283   

                AU09      AU10      AU12      AU14      AU15      AU17  \
label        

            Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                      
question1     0.005215  0.254841  0.297008  0.273977  0.034146  0.135140   
question2     0.002461  0.097293  0.120605  0.179106  0.098693  0.054595   
question3     0.004341  0.102454  0.118311  0.135786  0.065325  0.091110   
question4          NaN       NaN       NaN       NaN       NaN       NaN   
question5     0.001019  0.116756  0.067531  0.099316  0.038245  0.025901   
question6     0.003863  0.211959  0.260385  0.111905  0.070314  0.037119   
question7     0.001180  0.135431  0.155913  0.318191  0.056476  0.036753   
question8     0.006265  0.134904  0.318259  0.302291  0.085393  0.048996   
question9     0.002885  0.085842  0.198253  0.248262  0.052172  0.061356   
question10    0.001466  0.081661  0.082660  0.098990  0.051316  0.041936   

                AU09      AU10      AU12      AU14      AU15      AU17  \
label        

In [17]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

# 削除する列を指定
columns_to_drop = ['Confidence','label', 'AU07', 'AU11', 'AU20']

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # SVMモデルの初期化
    model = SVC(kernel='rbf', probability=True)# 'rbf'カーネルを使用（非線形データの場合）
    model.fit(X_train_combined, y_train_combined)
    
    # 予測
    y_pred = model.predict(X_test)
    y_pred_prob = model.predict_proba(X_test)[:, 1]  # クラス1の確率
    
    # 精度計算
    accuracy = accuracy_score(y_test, y_pred) * 100  # 正答率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")  # 小数点以下2桁で%表示
    print(f"正解ラベル: {y_test.tolist()}")  # 正解ラベルをリストとして表示
    print(f"予測ラベル: {y_pred.tolist()}")  # 予測ラベルをリストとして表示
    print(f"クラス1の確率: {y_pred_prob}")  # 各テストデータ点のクラス1の確率

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")


===== テストデータ: ファイル 1/31 =====
ファイル名: filtered_ID30_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.38181827 0.24921474 0.3476505  0.25002878 0.22496225 0.20071484
 0.26377844 0.29263315 0.2989528  0.39030307]

===== テストデータ: ファイル 2/31 =====
ファイル名: filtered_ID31_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.25059893 0.23528723 0.44335315 0.29113467 0.28621716 0.36542917
 0.33906527 0.33790511 0.36063559 0.43574903]

===== テストデータ: ファイル 3/31 =====
ファイル名: filtered_ID32_smoothed.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.22843053 0.18221583 0.16609252 0.22747487 0.29842796 0.21239977
 0.17798897 0.18921312 0.18740886]

===== テストデータ: ファイル 4/31 =====
ファイル名: filtered_ID33_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.20627721 0.36377173 0.311

In [15]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report

# 削除する列を指定
columns_to_drop = ['Confidence', 'label', 'AU07', 'AU11', 'AU20']

# グリッドサーチ用のパラメータ設定
param_grid = {
    'C': [0.1, 1, 10, 100],         # 正則化パラメータ
    'gamma': [1, 0.1, 0.01, 0.001], # RBFカーネルのgamma
    'kernel': ['rbf']               # カーネルはRBF
}

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # SVMモデルのグリッドサーチ
    model = SVC(probability=True)
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=3, n_jobs=-1)
    grid_search.fit(X_train_combined, y_train_combined)
    
    # 最適なモデルを取得
    best_model = grid_search.best_estimator_
    print(f"最適パラメータ: {grid_search.best_params_}")
    
    # テストデータで予測
    y_pred = best_model.predict(X_test)
    y_pred_prob = best_model.predict_proba(X_test)[:, 1]  # クラス1の確率
    
    # 精度計算
    accuracy = accuracy_score(y_test, y_pred) * 100  # 正答率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")
    print(f"正解ラベル: {y_test.tolist()}")
    print(f"予測ラベル: {y_pred.tolist()}")
    print(f"クラス1の確率: {y_pred_prob}")
    
# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")



===== テストデータ: ファイル 1/31 =====
最適パラメータ: {'C': 10, 'gamma': 0.1, 'kernel': 'rbf'}
ファイル名: filtered_ID30_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.48775083 0.33206316 0.3845824  0.3610118  0.26894888 0.32094642
 0.39800792 0.39104412 0.38483529 0.43851334]

===== テストデータ: ファイル 2/31 =====
最適パラメータ: {'C': 0.1, 'gamma': 1, 'kernel': 'rbf'}
ファイル名: filtered_ID31_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.33214544 0.32248481 0.47642388 0.31071906 0.31634699 0.37268057
 0.34002012 0.46562585 0.37022616 0.37562792]

===== テストデータ: ファイル 3/31 =====
最適パラメータ: {'C': 0.1, 'gamma': 1, 'kernel': 'rbf'}
ファイル名: filtered_ID32_smoothed.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.34287548 0.24404838 0.20168413 0.27158799 0.27114647 0.21264399
 0.2278507  0.23843924 0.22197327]

===== テストデータ: ファイル 4/31 =====
最適パラメータ: {'C

最適パラメータ: {'C': 0.1, 'gamma': 1, 'kernel': 'rbf'}
ファイル名: filtered_ID69_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.36601641 0.2752061  0.35386742 0.24455295 0.35293473 0.29558997
 0.22271771 0.31916524 0.29413428 0.27253503]

===== テストデータ: ファイル 28/31 =====
最適パラメータ: {'C': 0.1, 'gamma': 1, 'kernel': 'rbf'}
ファイル名: filtered_ID70_smoothed.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.86774584 0.548384   0.35773187 0.63867657 0.49358339 0.4137266
 0.84598227 0.5        0.67334494]

===== テストデータ: ファイル 29/31 =====
最適パラメータ: {'C': 0.1, 'gamma': 1, 'kernel': 'rbf'}
ファイル名: filtered_ID71_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.41260366 0.30163708 0.3216208  0.29490047 0.40055579 0.28821968
 0.31668231 0.30636667 0.37797129 0.29426278]

===== テストデータ: ファイル 30/31 =====
最適パラメータ: {'C': 0.1, 'gamma': 1, 'kernel': 

## SVM（フィルタ後の先頭10フレームの最大値と最小値の差分）

In [16]:
# 処理するフォルダのパス
folder_path = './output_moving_filtered_data'

# フォルダ内のすべてのCSVファイルを取得
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
labels = [0,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,1,1,1,1,1,0,1,0,0,0]
i = 0
# 各ファイルに対して処理を適用
result_list = []  # 平均値を保存するリスト

for file_name in csv_files:
    file_path = os.path.join(folder_path, file_name)
    
    # データの読み込み
    df = pd.read_csv(file_path, low_memory=False)
    
    # `question1`から`question10`のみをフィルタリング
    filtered_df = df[df["label"].str.match(r'^question(10|[1-9])$', na=False)]
    
    # Confidenceが0.90以上の行を抽出
    filtered_df = filtered_df[filtered_df["Confidence"] >= 0.90]
    
    # `Confidence`列と`AU`で始まる列を抽出
    columns_to_extract = ["label", "Confidence"] + [col for col in filtered_df.columns if col.startswith("AU")]
    filtered_df = filtered_df[columns_to_extract]
    
    # 順序を保つためのカスタム順序のカテゴリを定義
    filtered_df["label"] = pd.Categorical(
        filtered_df["label"],
        categories=[f"question{i}" for i in range(1, 11)],
        ordered=True
    )
    
    # ラベルごとに最初の10フレームを取得
    first_10_frames_per_label = (
        filtered_df.groupby("label")
        .apply(lambda group: group.head(10))
        .reset_index(drop=True)  # グループ化インデックスをリセット
    )

    # ラベルごとに最大値を計算
    max_values_per_label = (
        first_10_frames_per_label.groupby("label").max(numeric_only=True)
    )

    # ラベルごとに最小値を計算
    min_values_per_label = (
        first_10_frames_per_label.groupby("label").min(numeric_only=True)
    )

    # 最大値と最小値の差分を計算
    range_values_per_label = max_values_per_label - min_values_per_label

    
    # 結果をリストに保存
    result_list.append((file_name, range_values_per_label))

    # Yを追加
    range_values_per_label["Y"] = labels[i]
    i = i + 1
    
# 各ファイルの結果を出力
for file_name, mean_values in result_list:
    print(f"Results for {file_name}:")
    print(mean_values)
    print()

Results for filtered_ID30_smoothed.csv:
            Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                      
question1     0.003822  0.036655  0.030980  0.085982  0.017956  0.038607   
question2     0.003976  0.065868  0.105564  0.091642  0.044809  0.028340   
question3     0.002594  0.026349  0.062172  0.042632  0.047751  0.019277   
question4     0.002409  0.052017  0.049216  0.049436  0.070426  0.030970   
question5     0.005654  0.017081  0.084297  0.122237  0.016569  0.076541   
question6     0.005794  0.008200  0.029056  0.111280  0.038556  0.048539   
question7     0.003267  0.039577  0.055206  0.088458  0.086344  0.024362   
question8     0.002394  0.024537  0.059238  0.144408  0.022217  0.015875   
question9     0.002268  0.055435  0.065355  0.022176  0.094148  0.021973   
question10    0.003167  0.017304  0.039235  0.068160  0.050081  0.017603   

                AU09      AU10      AU12      A

            Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                      
question1     0.001619  0.092692  0.160036  0.128746  0.023610  0.079133   
question2     0.000043  0.012814  0.072197  0.066077  0.083080  0.025699   
question3     0.000084  0.012334  0.021817  0.053899  0.042007  0.011550   
question4     0.000056  0.017870  0.068477  0.103819  0.158005  0.022459   
question5     0.000041  0.020753  0.079470  0.033860  0.008601  0.049770   
question6     0.000075  0.047111  0.139938  0.056881  0.010213  0.016802   
question7     0.000026  0.011119  0.048550  0.116847  0.031763  0.037214   
question8     0.000418  0.022929  0.068885  0.072880  0.012237  0.020171   
question9     0.000098  0.021537  0.056024  0.103756  0.030245  0.044903   
question10    0.000294  0.040601  0.035419  0.055750  0.050002  0.055044   

                AU09      AU10      AU12      AU14      AU15      AU17  \
label        

            Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                      
question1          NaN       NaN       NaN       NaN       NaN       NaN   
question2     0.053148  0.030646  0.020718  0.116575  0.015676  0.026828   
question3          NaN       NaN       NaN       NaN       NaN       NaN   
question4     0.009893  0.153194  0.204256  0.227073  0.014691  0.005536   
question5     0.023960  0.055192  0.091173  0.105020  0.024473  0.031447   
question6     0.014196  0.026350  0.033633  0.060231  0.018756  0.017427   
question7          NaN       NaN       NaN       NaN       NaN       NaN   
question8     0.010923  0.043665  0.105376  0.151348  0.016104  0.038146   
question9     0.025861  0.019988  0.039782  0.046027  0.011872  0.010407   
question10    0.028938  0.040390  0.065478  0.101305  0.017767  0.036671   

                AU09      AU10      AU12      AU14      AU15      AU17  \
label        

In [19]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

# 削除する列を指定
columns_to_drop = ['Confidence','label', 'AU07', 'AU11', 'AU20']

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # SVMモデルの初期化
    model = SVC(kernel='rbf', probability=True)# 'rbf'カーネルを使用（非線形データの場合）
    model.fit(X_train_combined, y_train_combined)
    
    # 予測
    y_pred = model.predict(X_test)
    y_pred_prob = model.predict_proba(X_test)[:, 1]  # クラス1の確率
    
    # 精度計算
    accuracy = accuracy_score(y_test, y_pred) * 100  # 正答率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")  # 小数点以下2桁で%表示
    print(f"正解ラベル: {y_test.tolist()}")  # 正解ラベルをリストとして表示
    print(f"予測ラベル: {y_pred.tolist()}")  # 予測ラベルをリストとして表示
    print(f"クラス1の確率: {y_pred_prob}")  # 各テストデータ点のクラス1の確率

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")


===== テストデータ: ファイル 1/31 =====
ファイル名: filtered_ID30_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.41766335 0.4010005  0.33600522 0.26258443 0.35878583 0.35248817
 0.35120323 0.5        0.27904259 0.35045134]

===== テストデータ: ファイル 2/31 =====
ファイル名: filtered_ID31_smoothed.csv
予測率: 90.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
クラス1の確率: [0.31537834 0.29524744 0.43916377 0.38136651 0.32985903 0.32750298
 0.41603034 0.50793422 0.3528734  0.33931253]

===== テストデータ: ファイル 3/31 =====
ファイル名: filtered_ID32_smoothed.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.35189463 0.26808355 0.23966296 0.25720214 0.32232138 0.27625881
 0.26161037 0.25880466 0.28721513]

===== テストデータ: ファイル 4/31 =====
ファイル名: filtered_ID33_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.28227947 0.34212552 0.3081

In [17]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report

# 削除する列を指定
columns_to_drop = ['Confidence', 'label', 'AU07', 'AU11', 'AU20']

# グリッドサーチ用のパラメータ設定
param_grid = {
    'C': [0.1, 1, 10, 100],         # 正則化パラメータ
    'gamma': [1, 0.1, 0.01, 0.001], # RBFカーネルのgamma
    'kernel': ['rbf']               # カーネルはRBF
}

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # SVMモデルのグリッドサーチ
    model = SVC(probability=True)
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=3, n_jobs=-1)
    grid_search.fit(X_train_combined, y_train_combined)
    
    # 最適なモデルを取得
    best_model = grid_search.best_estimator_
    print(f"最適パラメータ: {grid_search.best_params_}")
    
    # テストデータで予測
    y_pred = best_model.predict(X_test)
    y_pred_prob = best_model.predict_proba(X_test)[:, 1]  # クラス1の確率
    
    # 精度計算
    accuracy = accuracy_score(y_test, y_pred) * 100  # 正答率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")
    print(f"正解ラベル: {y_test.tolist()}")
    print(f"予測ラベル: {y_pred.tolist()}")
    print(f"クラス1の確率: {y_pred_prob}")
    
# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")



===== テストデータ: ファイル 1/31 =====
最適パラメータ: {'C': 0.1, 'gamma': 1, 'kernel': 'rbf'}
ファイル名: filtered_ID30_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.39892937 0.38674898 0.372059   0.36728016 0.38659235 0.38697358
 0.37828561 0.39266797 0.37746033 0.37947525]

===== テストデータ: ファイル 2/31 =====
最適パラメータ: {'C': 0.1, 'gamma': 1, 'kernel': 'rbf'}
ファイル名: filtered_ID31_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.35896787 0.36764113 0.41529367 0.37737181 0.36503338 0.36055038
 0.36944816 0.41589533 0.38903142 0.37362044]

===== テストデータ: ファイル 3/31 =====
最適パラメータ: {'C': 10, 'gamma': 0.1, 'kernel': 'rbf'}
ファイル名: filtered_ID32_smoothed.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.36518823 0.30043364 0.31816664 0.32037357 0.33395833 0.33384333
 0.3008085  0.29455366 0.34904367]

===== テストデータ: ファイル 4/31 =====
最適パラメータ: {'C

最適パラメータ: {'C': 10, 'gamma': 0.1, 'kernel': 'rbf'}
ファイル名: filtered_ID69_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.36314056 0.37909375 0.38782953 0.34074221 0.41400141 0.35368571
 0.31511818 0.38350455 0.3388584  0.36333673]

===== テストデータ: ファイル 28/31 =====
最適パラメータ: {'C': 0.1, 'gamma': 1, 'kernel': 'rbf'}
ファイル名: filtered_ID70_smoothed.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.37768962 0.38884694 0.36420188 0.42213994 0.38869662 0.34676303
 0.45429763 0.3722038  0.40072338]

===== テストデータ: ファイル 29/31 =====
最適パラメータ: {'C': 10, 'gamma': 0.1, 'kernel': 'rbf'}
ファイル名: filtered_ID71_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.34741308 0.33377867 0.34420298 0.34601855 0.37281562 0.34046226
 0.36357703 0.37591065 0.35734275 0.35289427]

===== テストデータ: ファイル 30/31 =====
最適パラメータ: {'C': 100, 'gamma': 0.01, 'ker